# Visualize MuData in Vitessce

In [ ]:
!lamin load laminlabs/lamindata

In [ ]:
import lamindb as ln

ln.context.uid = "aCrfeVoAaDr90000"
ln.context.track()

## Prepare dataset

Load MuData object from: https://lamin.ai/laminlabs/lamindata/artifact/DDFxKYXoNMmHzQAHqpx0

In [ ]:
mdata_artifact = ln.Artifact.get("DDFxKYXo")
mdata = mdata_artifact.load()
mdata

Subset to hvg for visualization:

In [ ]:
mdata.mod["rna"] = mdata["rna"][:, mdata["rna"].var["highly_variable"]]
mdata.update()

Write to `anndata.zarr`:

In [ ]:
output_path = "mudata_papalexi21.anndata.zarr"
mdata.write_zarr(output_path)

## Save dataset

In [ ]:
zarr_artifact = ln.Artifact(
    output_path,
    description="MuData from Papalexi21",
)
zarr_artifact.save()

In [ ]:
from vitessce import (
    VitessceConfig,
    Component as cm,
    AnnDataWrapper,
)

vc = VitessceConfig(
    schema_version="1.0.15",
    description=zarr_artifact.description,
)
dataset = vc.add_dataset(name="Papalexi21").add_object(
    AnnDataWrapper(
        adata_url=zarr_artifact.path.to_url(),
        obs_feature_matrix_path="mod/rna/X",
        obs_embedding_paths=["mod/rna/obsm/X_umap", "mod/rna/obsm/X_pca"],
        obs_embedding_names=["UMAP", "PCA"],
        obs_set_paths=[
            "mod/rna/obs/gene_target",
            "mod/rna/obs/guide_ID",
            "mod/rna/obs/Phase",
            "mod/rna/obs/replicate",
        ],
        obs_set_names=["Gene target", "Guide id", "Cell cycle", "Replicate"],
    )
)

obs_sets = vc.add_view(cm.OBS_SETS, dataset=dataset)
obs_sets_sizes = vc.add_view(cm.OBS_SET_SIZES, dataset=dataset)
umap = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="UMAP")
pca = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="PCA")
heatmap = vc.add_view(cm.HEATMAP, dataset=dataset)
genes = vc.add_view(cm.FEATURE_LIST, dataset=dataset)
vc.layout(((umap | pca) / heatmap) | ((genes | obs_sets) / obs_sets_sizes))

# inspect the config
vc.to_dict()

In [ ]:
from lamindb.integrations import save_vitessce_config

vc_artifact = save_vitessce_config(vc, description="View Papalexi21 in Vitessce")

In [ ]:
ln.finish()